In [4]:
import re

In [ ]:
class Lexer:
    def __init__(self,source_code):
        self.source_code = source_code
        self.tokens = []

    def tokenize(self):
        token_specification = [
            ("FUNCTION", r'fn'),
            ("MAIN", r'main'),
            ("LET", r'let'),
            ("INT", r'int'),
            ("CHAR", r'char'),
            ("FLOAT", r'float'),
            ("IF", r'if'),
            ("ELSE", r'else'),
            ("WHILE", r'while'),
            ("PRINTLN", r'println'),
            ("RETURN", r'return'),
            ("LBRACKET", r'('),
            ("RBRACKET", r')'),
            ("LBRACE", r'{'),
            ("RBRACE", r'}'),
            ("ARROW", r'->'),
            ("COLON", r':'),
            ("SEMICOLON", r';'),
            ("COMMA", r','),
            ("ASSIGN", r'='),
            ("EQ", r'=='),
            ("NE", r'!='),
            ("GT", r'>'),
            ("GE", r'>='),
            ("LT", r'<'),
            ("LE", r'<='),
            ("PLUS", r'+'),
            ("MINUS", r'-'),
            ("MULT", r'*'),
            ("DIV", r'/'),
            ("ID", r'[a-zA-Z]([a-zA-Z0-9_])*'),
            ("INT_CONST", r'[0-9]([0-9])*'),
            ("FLOAT_CONST", r'[0-9]([0-9])*.[0-9]([0-9])*)'),
            ("CHAR_LITERAL", r'Σ'),
            ("FMT_STRING", r'Σ+'),
            ("MISMATCH", r'.')
            ]

        tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
        for mo in re.finditer(tok_regex, self.source_code):
            kind = mo.lastgroup
            value = mo.group()
            if kind == "SKIP":
                continue
            elif kind == "MISMATCH":
                raise RuntimeError(f'Unexpected character: {value}')
            else:
                self.tokens.append((kind, value))

